In [1]:
import numpy as np
import random
import cv2

from collections import namedtuple, deque
import math
import torch

''' Replay Buffer
'''
N_CHANNEL, N_HIGH, N_WEIGHT = 50, 256, 256
# ReplayMemory의 역할은 state, action, reward, next_state를 저장하는 것
# state, action, reward, next_state를 저장해서 학습할 때 사용함.
class ReplayMemory():
    def __init__(self, memory_size):
        # self.memory의 역할은 state, action, reward, next_state를 저장하는 것
        self.memory_counter = 0
        self.memory_size = memory_size
        self.state_memory = torch.FloatTensor(self.memory_size, 1, N_CHANNEL, N_HIGH, N_WEIGHT)
        self.action_memory = torch.LongTensor(self.memory_size)
        self.reward_memory = torch.FloatTensor(self.memory_size)
        self.state__memory = torch.FloatTensor(self.memory_size, 1, N_CHANNEL, N_HIGH, N_WEIGHT)

    def store(self, s, a, r, s_):
        # 여기를 보면 index에 맞게 "순서대로" 저장하는 것을 볼 수 있음.
        index = self.memory_counter % self.memory_size
        self.state_memory[index] = s
        self.action_memory[index] = torch.LongTensor([a.tolist()])
        self.reward_memory[index] = torch.FloatTensor([r])
        self.state__memory[index] = s_
        self.memory_counter += 1

    def sample(self, size):
        sample_index = np.random.choice(self.memory_size, size)
        state_sample = torch.FloatTensor(size,1,  N_CHANNEL, N_HIGH, N_WEIGHT)
        action_sample = torch.LongTensor(size, 1)
        reward_sample = torch.FloatTensor(size, 1)
        state__sample = torch.FloatTensor(size, 1, N_CHANNEL, N_HIGH, N_WEIGHT)
        for index in range(sample_index.size):
            state_sample[index] = self.state_memory[sample_index[index]]
            action_sample[index] = self.action_memory[sample_index[index]]
            reward_sample[index] = self.reward_memory[sample_index[index]]
            state__sample[index] = self.state__memory[sample_index[index]]
        return state_sample, action_sample, reward_sample, state__sample


''' Reshape state patches to full size input 50 x 256 x 256
'''
# crop_reshape 함수는 3D image를 (channel, 256, 256)으로 확장시키는 함수
def crop_reshape(img, x,y,z,w,h,d):
    img = np.array(img)
    new_image = []
    image_patch = img[z:z+d, x:x+h, y:y+w]
    for i in range(image_patch.shape[0]):
        new_image.append(cv2.resize(image_patch[i], (256,256)))
    new_image = np.array(new_image)
    return torch.tensor(new_image.reshape(1,1,new_image.shape[0], new_image.shape[1], new_image.shape[2])).float()

'''Compute Accuracy for Training 
'''

# binary_acc함수는 y_pred와 y_test를 받아서 정확도를 계산하는 함수
def binary_acc(y_pred, y_test):
    #y_pred[y_pred >= 0.5] = 1 
    #y_pred[y_pred < 0.5] = 0
    _, y_pred = y_pred.max(1)
    correct_results_sum = (y_pred == y_test).sum().float()
    acc = correct_results_sum#/y_test.shape[0]
    acc = torch.round(acc)

    return acc

# 3Dmodels  

<img src="./src/DQN_based_Localization_figure.png" alt="Description of the image" width="600" height="400"/>

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d,MaxUnpool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from skimage import util
from torchvision import transforms, models
from torch import optim

import numpy as np
import random
import torchvision
from collections import namedtuple, deque
import math

# DQNSolver함수는 3D image를 받아서 7개의 action을 출력하는 DQN 모델을 생성하는 함수
# DQN이란 Deep Q Network의 약자로 Q-learning을 딥러닝으로 구현한 것
class DQNSolver(nn.Module):
    """
    Convolutional Neural Net with 3 conv3D layers and two linear layers
    """
    def __init__(self, input_shape, n_actions):
        super(DQNSolver, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(input_shape[0], 32, kernel_size=(5,5,5), stride=2),
            nn.LeakyReLU(),
            nn.MaxPool3d((2, 2, 2)),
            nn.Conv3d(32, 64, kernel_size=(5,5,5), stride=1),
            nn.LeakyReLU(),
            nn.MaxPool3d((1, 2, 2)),
            nn.Conv3d(64, 64, kernel_size=(3,3,3), stride=2),
            nn.LeakyReLU(),
            nn.MaxPool3d((2, 2, 2)),
        )
        
        # conv_out_size의 역할은 conv layer를 거치고 나온 output의 size를 계산하는 것
        # _get_conv_out에 어떤 shape이 들어가냐면, 
        # DQNSolver의 예상 input_shape의 크기가 (1,50,256,256)라면, self.conv이후의 shape은  (64, 1, 7, 7)이므로
        # conv_out_size는 64*1*7*7 = 3136이 됨.
        # 이걸 self.fc layer를 이용해 7개로 줄여 action을 결정하게 됨.
        # 여기서 7개의 액션은 각각의 방향으로 움직이는 것을 의미함.
        # 0 - up, 1- down, 2 - right, 3- left, 4- top, 5 - bottom, 6 - terminate
        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions)
        )

    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)


class Classifier(nn.Module):
    """
    Convolutional Neural Net with 3 conv3D layers and two linear layers
    """
    def __init__(self, input_shape, num_features):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(input_shape[0], 32, kernel_size=(5,5,5), stride=2),
            nn.LeakyReLU(),
            nn.MaxPool3d((2, 2, 2)),
            nn.Conv3d(32, 64, kernel_size=(5,5,5), stride=1),
            nn.LeakyReLU(),
            nn.MaxPool3d((1, 2, 2)),
            nn.Conv3d(64, 64, kernel_size=(3,3,3), stride=2),
            nn.LeakyReLU(),
            nn.MaxPool3d((2, 2, 2)),
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 256),
            nn.ReLU(),
            nn.Linear(256, num_features),
            nn.ReLU()
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)

# concat_
class concat_classifier(nn.Module):
    def __init__(self, input_shape1,input_shape2, classes):
        super(concat_classifier, self).__init__()

        input_shape = input_shape1[1] + input_shape2[1]
        self.fc = nn.Sequential(
            nn.Linear(input_shape, 512),
            nn.ReLU(),
            nn.Linear(512, classes),
            nn.Sigmoid()
            )

    def forward(self,x1, x2):
        cat_out = torch.cat((x1,x2), dim = 1)
        output = self.fc(cat_out)
        return output


class combine_model(nn.Module):
    def __init__(self, model1, model2, model3):
        super(combine_model, self).__init__()

        self.model1 = model1
        self.model2 = model2
        self.model3 = model3

    def forward(self, x1, x2):
        input1 = x1
        input2 = x2

        out1 = self.model1(input1)
        out2 = self.model2(input2)
        pred = self.model3(out1, out2)

        return pred

# 우리 데이터의 input shape이 어떻게 되는지 확인해야함
policy_model = DQNSolver((1,50,256,256), 7)
target_model = DQNSolver((1,50,256,256), 7)
# hippocampus는 3D shape이 50, 256, 256인 이미지를 받아서 512개의 feature로 변환
# hippocampus란 해마를 의미함.
classifier_hippocampus = Classifier((1,50,256,256), 512)
# global은 3D shape이 145, 256, 256인 이미지를 받아서 1024개의 feature로 변환
# TODO: 둘 차이가 많이 나는데 이게 맞는건지 확인
classifier_global = Classifier((1,145,256,256), 1024)
final_model = concat_classifier((1,512), (1, 1024), 2)
# final_model이 2개의 feature를 받아서 2개의 class로 분류하는 모델
# 현재의 프로젝트 맥락에서 각각은 AD, CN을 의미함.
# 여기서 AD는 Alzheimer's Disease, CN은 Control을 의미함.
class_model = combine_model(classifier_hippocampus, classifier_global, final_model)

# DQN_agent

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d,MaxUnpool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from skimage import util
from torchvision import transforms, models
from torch import optim

from medpy.io import load
import cv2

''' DQN Agent
'''
class DQNAgent(object):
    def __init__(self, policy_model, target_model, replay_memory):
            self.batch_size = 100
            self.gamma = 0.99
            self.eps = 1
            self.target_update = 0
            screen_height, screen_width, screen_depth = 256, 256, 145
            self.n_actions = 7
            self.policy_model = policy_model
            self.target_model = target_model
            self.target_model.eval()
            self.optimizer = optim.Adam(self.policy_model.parameters(), lr = 0.00009)
            self.loss_func = torch.nn.SmoothL1Loss()#nn.MSELoss()
            self.memory = replay_memory

    ''' Compute reward with L2 distance between centers of state and gd
    '''
    def compute_reward(self, actual_state, prev_state, ground_truth, threshold):
            x,y,z,w,h,d = actual_state
            x_p,y_p,z_p,w_p,h_p,d_p = prev_state
            x_gd, y_gd, z_gd, w_gd, h_gd, d_gd = ground_truth

            center = np.array([(x+w)/2,(y+h)/2, (z+d)/2])
            center_gd = np.array([(x_gd + w_gd)/2,(y_gd + h_gd)/2, (z_gd + d_gd)/2])
            center_p = np.array([(x_p + w_p)/2,(y_p + h_p )/2, (z_p + d_p)/2])
            dist_l2 = np.linalg.norm(center - center_gd)
            dist_l2_p = np.linalg.norm(center_p - center_gd)

            if dist_l2 < threshold:
                game = "END"
                reward = 100
                return game, reward
            else:
                game = "continue"
                reward = min(1, -dist_l2) #dist_l2 - dist_l2_p #
                return game, reward

    ''' 0 - up, 1- down, 2 - right, 3- left, 4- top, 5 - bottom, 6 - terminate
    '''
    # def next_state(self, prev_state, actn, step):
    #     # TODO: 일단 우리 data는 240x240x155이므로 그에 맞게 코드를 수정하던가 우리의 데이터를 interpolation해야할 듯 함.
    #     # 근데 이미 interpolation하는 코드가 있어서, 그걸 사용하면 될 듯함.
    #         x,y,z,w,h,d = prev_state
    #         # max_x:가 왜 255인지 모르겠음
    #         max_x = 255
    #         max_y = 255
    #         max_z = 140
    #         min_x = 0
    #         min_y = 0
    #         min_z = 0

    #         if actn == 0:
    #             if x + w + step >= 255:
    #                 x = 100
    #             else:
    #                 x = x + step
    #         elif actn == 1:
    #             if x - step <= 0:
    #                 x = 100
    #             else:
    #                 x = x - step
    #         elif actn == 2:
    #             if y + h + step >= 255:
    #                 y = 100
    #             else:
    #                 y = y + step
    #         elif actn == 3:
    #             if y - step <= 0:
    #                 y = 100
    #             else:
    #                 y = y - step
    #         elif actn == 4:
    #             if z + d + step >= 144 :
    #                 z = 50
    #             else:
    #                 z = z + step
    #         elif actn == 5:
    #             if z - step <= 0:
    #                 z = 50
    #             else:
    #                 z = z - step

    #         return [int(x),int(y),int(z),int(w),int(h),int(d)]
    def next_state(self, prev_state, actn, step):
        # Constants for boundaries
        max_bounds = {'x': 255, 'y': 255, 'z': 140}
        min_bounds = {'x': 0, 'y': 0, 'z': 0}
        
        x, y, z, w, h, d = prev_state

        # Action mapping
        actions = {
            0: ('x', step, max_bounds['x'], 100),
            1: ('x', -step, min_bounds['x'], 100),
            2: ('y', step, max_bounds['y'], 100),
            3: ('y', -step, min_bounds['y'], 100),
            4: ('z', step, max_bounds['z'], 50),
            5: ('z', -step, min_bounds['z'], 50)
        }
        
        # Get the corresponding parameters for the action
        axis, movement, bound, reset_value = actions.get(actn, (None, 0, 0, 0))
        
        # Apply the movement and boundary checks
        if axis == 'x':
            x = self.update_coordinate(x, movement, bound, reset_value)
        elif axis == 'y':
            y = self.update_coordinate(y, movement, bound, reset_value)
        elif axis == 'z':
            z = self.update_coordinate(z, movement, bound, reset_value)
        
        return [int(x),int(y),int(z),int(w),int(h),int(d)]

    def update_coordinate(self, coord, movement, bound, reset_value):
        new_coord = coord + movement
        if movement > 0:  # Moving forward
            if new_coord >= bound:
                return reset_value
        else:  # Moving backward
            if new_coord <= bound:
                return reset_value
        return new_coord


    ''' Start with high eps then reduce eps, if game = END then action should be 6
    '''
    # epsilon-greedy 방법을 이용해서 action을 선택하는 함수
    def select_action(self, state, game, eps ):
            actn = 6
            if game == "continue":
                sample = random.random()
                if sample < eps:
                    actn = np.asarray(random.randrange(6))

                else:
                    out = self.policy_model(state)
                    _, actn = torch.max(out.data, 1)

                actn = np.array(actn)
                return actn
            else:
                actn = np.array(actn)
                return actn

    def select_action_test(self, state):
        out = self.target_model(state)
        _, actn = torch.max(out.data,1)
        return actn

    def store_transition(self, s, a, r, s_):
            self.memory.store(s, a, r, s_)

    ''' Optimize model with replay memory
    '''

    def optimize(self, counter):

            if self.target_update > counter:
                self.target_model.load_state_dict(self.policy_model.state_dict())
                self.target_update = 0
                #print("Updating Target Net")

            self.target_update += 1


            s_s, a_s, r_s, s__s = self.memory.sample(self.batch_size)

            q_eval = self.policy_model(s_s).gather(1, a_s)
            q_next = self.target_model(s__s).detach()
            q_target = r_s + self.gamma * q_next.max(1)[0].view(self.batch_size, 1)
            loss = self.loss_func(q_eval, q_target)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            return loss

my_dqn = DQNAgent(policy_model, target_model, ReplayMemory(1000))

# Custom Data Gen

In [4]:
DATA_PATH = '../archive_BraTS2022'
TRAIN_PATH = f'{DATA_PATH}/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
TEST_PATH = f'{DATA_PATH}/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'
DATA_TYPES = ['flair', 't1', 't1ce', 't2', 'seg']
N_FOLDS = 5

In [12]:
import os
import nibabel as nib
from glob import glob
import re
from tqdm.auto import tqdm

def load_brats_data(path):
    print('load_brats_data...')
    files_path = []
    mask_paths = []
    unique_id = []
    for file_path in glob(f'{path}/*/*.nii'):
        # t1 파일만 처리
        if re.search(r"_t1(?=\.)", file_path):
            files_path.append(file_path)
        # seg 파일만 처리
        if "seg" in file_path:  # segmentation mask file
            mask_paths.append(file_path)
        unique_id.append(str(int(file_path.split('_')[-2])))
        
    return sorted(files_path), sorted(mask_paths), sorted(list(set(unique_id)), key=lambda x: int(x))

var1 = 0
var2 = 150
def compute_brats_masks(mask_paths, var1, var2):
    print('compute_brats_masks...')
    all_masks = []
    for mask_path in tqdm(mask_paths):
        mask = nib.load(mask_path).get_fdata()  # 마스크 데이터 로드
        dummy_var = []
        for j in range(min(var2, mask.shape[-1])):
            if j <= var2:  # 주어진 범위 내의 슬라이스만 처리
                msk = mask[:, :, j] / mask.max() * 255  # 슬라이스 정규화 및 스케일링(0 ~ 1)
                # msk = cv2.resize(msk, (256, 256))  # 이미지 크기 조정
                _, bw_img = cv2.threshold(msk.astype(np.uint8), 10, 255, cv2.THRESH_OTSU)  # 이진화
                contours, _ = cv2.findContours(bw_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)  # 컨투어 찾기
                
                # 가장 큰 컨투어 찾기
                max_area = 0
                max_rect = (0, 0, 0, 0)
                for contour in contours:
                    rect = cv2.boundingRect(contour)
                    area = rect[2] * rect[3]  # rectangle의 넓이 (width * height)
                    if area > max_area:
                        max_area = area
                        max_rect = rect
                
                # 가장 큰 컨투어의 사각형 정보 저장
                if max_area > 0:
                    dummy_var.append([j, max_rect])
                else:
                    dummy_rect = [0, 0, 0, 0]  # 컨투어가 없을 경우
                    dummy_var.append([j, dummy_rect])
        all_masks.append(dummy_var)
    
    np_all_masks = np.array(all_masks, dtype=object)
    print(np_all_masks.shape)
    return all_masks


def brats_markers(slices_mask):
    print('brats_markers...')
    marks = []
    for i in tqdm(range(len(slices_mask))):
        x_min = []
        y_min = []
        w_max = []
        h_max = []
        indx_diff = []
        for j in range(len(slices_mask[i])):
            x_min.append(slices_mask[i][j][1][0])
            y_min.append(slices_mask[i][j][1][1])
            w_max.append(slices_mask[i][j][1][2])
            h_max.append(slices_mask[i][j][1][3])

            if slices_mask[i][j][1][0] != 0:
                indx_diff.append(slices_mask[i][j][0])
        x_min = np.array(x_min)
        y_min = np.array(y_min)
        x_mn = np.min(x_min[np.nonzero(x_min)])
        y_mn = np.min(y_min[np.nonzero(y_min)])
        w_mx = np.array(w_max).max()
        h_mx = np.array(h_max).max()
        dim = [x_mn, y_mn, w_mx, h_mx]
        d_mx = indx_diff[-1] - indx_diff[0]
        z_mn = indx_diff[0]
        marks.append([x_mn, y_mn, z_mn, w_mx, h_mx, d_mx])
    return marks

all_files_id, mask_paths, unique_id = load_brats_data(TRAIN_PATH)
sliced_masks = compute_brats_masks(mask_paths, var1, var2)
sliced_markers = brats_markers(sliced_masks)

sliced_masks
# # Create labels (example: 1 for tumor presence, 0 for no tumor)
# labels = np.ones((len(sliced_markers), 1))  # Assuming all masks are tumors

# label_data = np.hstack([sliced_markers, labels])


load_brats_data...
compute_brats_masks...


  0%|          | 0/369 [00:00<?, ?it/s]

(369, 150, 2)
brats_markers...


  0%|          | 0/369 [00:00<?, ?it/s]

[[[0, [0, 0, 0, 0]],
  [1, [0, 0, 0, 0]],
  [2, [0, 0, 0, 0]],
  [3, [0, 0, 0, 0]],
  [4, [0, 0, 0, 0]],
  [5, [0, 0, 0, 0]],
  [6, [0, 0, 0, 0]],
  [7, [0, 0, 0, 0]],
  [8, [0, 0, 0, 0]],
  [9, [0, 0, 0, 0]],
  [10, [0, 0, 0, 0]],
  [11, [0, 0, 0, 0]],
  [12, [0, 0, 0, 0]],
  [13, [0, 0, 0, 0]],
  [14, [0, 0, 0, 0]],
  [15, [0, 0, 0, 0]],
  [16, [0, 0, 0, 0]],
  [17, [0, 0, 0, 0]],
  [18, [0, 0, 0, 0]],
  [19, [0, 0, 0, 0]],
  [20, [0, 0, 0, 0]],
  [21, [0, 0, 0, 0]],
  [22, [0, 0, 0, 0]],
  [23, [0, 0, 0, 0]],
  [24, [0, 0, 0, 0]],
  [25, [0, 0, 0, 0]],
  [26, [0, 0, 0, 0]],
  [27, [0, 0, 0, 0]],
  [28, [0, 0, 0, 0]],
  [29, [0, 0, 0, 0]],
  [30, (107, 93, 2, 2)],
  [31, (100, 88, 11, 9)],
  [32, (98, 80, 19, 18)],
  [33, (98, 80, 20, 20)],
  [34, (96, 74, 22, 26)],
  [35, (96, 74, 23, 26)],
  [36, (95, 73, 26, 27)],
  [37, (91, 73, 32, 28)],
  [38, (90, 72, 35, 29)],
  [39, (89, 71, 36, 29)],
  [40, (89, 71, 37, 29)],
  [41, (89, 71, 37, 29)],
  [42, (85, 71, 43, 32)],
  [43, (85, 7

# Data_Prep

In [4]:
import glob
import numpy as np
import random
import fnmatch
import os
from PIL import Image
from matplotlib import pyplot as plt
import PIL
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

from bs4 import BeautifulSoup
from medpy.io import load
import cv2

from collections import namedtuple, deque
import math
from sklearn.utils import shuffle
from tqdm import tqdm

''' Read files for AD and CN
'''

# def mask_load(path):
    # files_id = []
    # mask = []
    # unique_id = []
    # for root, folder, file in os.walk(os.path.abspath(path)):
    #     for masks in fnmatch.filter(file,"*.nii"):
    #         y = os.path.join(root, masks).rsplit("\\")[6]
    #         uid = os.path.join(root, masks).rsplit("\\")[8]
    #         mask.append(os.path.join(root, masks))
    #         files_id.append(y)
    #         unique_id.append(uid)
    # return files_id, mask, unique_id

# path_ad = "Path to AD\\ADNI"
# all_files_ad_id, mask_ad, ad_unique_id = mask_load(path_ad)

# path_cn = "Path to \\CN\\ADNI"
# all_files_cn_id, mask_cn, cn_unique_id = mask_load(path_cn)


brats_file_paths, brats_mask_paths, brats_unique_id = load_brats_data(TRAIN_PATH)
#MP-RAGE_, SAG_MP-RAGE, MP-RAGE-, MPRAGE, MPRAGE_, MP-RAGE

# def data_load(path, file_id, unique_id):
#   file_path = []
#   f_path = []
#   for root, folder, file in os.walk(os.path.abspath(path)):
#     f_path = []
#     for filename in fnmatch.filter(file, "*.dcm" ):
#       id1 = os.path.join(root, filename).rsplit("\\")[6]
#       if id1 in file_id:
#           id2 = os.path.join(root, filename).rsplit("\\")[8]
#           if id2 in unique_id:
#               x = os.path.join(root, filename).rsplit("\\")[7]
#               if (x == "MP-RAGE_" or x == "SAG_MP-RAGE" or x == "MP-RAGE-" or
#                    x == "MPRAGE" or x == "MPRAGE_" or x == "MP-RAGE") :
#                        f_path.append(os.path.join(root, filename))

#     if len(f_path) != 0:
#         file_path.append(f_path)
#   return file_path

# all_files_ad = data_load(path_ad, all_files_ad_id, ad_unique_id )

# def data_load_cn(path, file_id, unique_id):
#   file_path = []
#   f_path = []
#   for root, folder, file in os.walk(os.path.abspath(path)):
#     f_path = []
#     for filename in fnmatch.filter(file, "*.dcm" ):
#       id1 = os.path.join(root, filename).rsplit("\\")[6]
#       if id1 in file_id:
#           id2 = os.path.join(root, filename).rsplit("\\")[8]
#           if id2 in unique_id:
#               x = os.path.join(root, filename).rsplit("\\")[7]
#               if ( x == "MP-RAGE_" or x == "MP-RAGE-" or x == "MPRAGE" or
#                    x == "SAG_MP-RAGE_" or x == "MPRAGE_" or x == "MP_RAGE" or
#                    x == "MP-RAGE" or x == "MP-RAGE_" or x == "MP-RAGE__SERIES_2_" or
#                    x == "SAG_MP-RAGE") :
#                        f_path.append(os.path.join(root, filename))

#     if len(f_path) != 0:
#         file_path.append(f_path)
#   return file_path

# all_files_cn = data_load_cn(path_cn, all_files_cn_id, cn_unique_id)

''' Choose slices 25 - 125
'''
# 원래는 slice마다 image가 있어서, 이걸 전부 붙이는 과정이 있었는 듯 함.
var1 = 15
var2 = 159
# def slice_select(var1, var2, file_path):
#     new_filepath = []
#     for i in range(len(file_path)):
#         dummy_var = []
#         for j in range(len(file_path[i])):
#             if j>=var1 and j<= var2:
#                 dummy_var.append(file_path[i][j])
#         new_filepath.append(dummy_var)
#     return new_filepath

# sliced_all_files_cn = slice_select(var1, var2, all_files_cn)
# sliced_all_files_ad = slice_select(var1, var2, all_files_ad)

''' mask selection
'''

def compute_all_masks(var1, var2, file):
    all_masks = []
    for i in range(len(file)):
        mask1,_ = load(file[i])
        dummy_var = []
        for j in range(mask1.shape[0]):
            if j >= var1 and j <= var2:
                msk = 255*mask1[j]/mask1[j].max()
                msk = cv2.resize(msk, (256,256))
                _, bw_img = cv2.threshold(msk.astype(np.uint8), 10, 255, cv2.THRESH_OTSU)
                # _, bw_img = cv2.threshold(masks[indx_mx], 127, 255, cv2.THRESH_BINARY)
                contours,_ = cv2.findContours(bw_img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
                if len(contours) != 0:
                    # TODO: contours에서 가장 큰 것을 rect에 저장되도록 코드 수정.
                    # 그래서 2D image마다 contours를 찾아서 저장하고 있음, 이를 수정해야함.
                    rect =  cv2.boundingRect(contours[0])
                    dummy_var.append([j, rect])
                else:
                    dummy_rect = [0,0,0,0]
                    dummy_var.append([j, dummy_rect])

        all_masks.append(dummy_var)
    return all_masks

def compute_all_brats_masks(var1, var2, file):
    all_masks = []
    for i in range(len(file)):
        mask1,_ = load(file[i])
        dummy_var = []
        for j in range(mask1.shape[0]):
            if j >= var1 and j <= var2:
                msk = 255*mask1[j]/mask1[j].max()
                msk = cv2.resize(msk, (256,256))
                _, bw_img = cv2.threshold(msk.astype(np.uint8), 10, 255, cv2.THRESH_OTSU)
                # _, bw_img = cv2.threshold(masks[indx_mx], 127, 255, cv2.THRESH_BINARY)
                contours,_ = cv2.findContours(bw_img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
                if len(contours) != 0:
                    # TODO: contours에서 가장 큰 것을 rect에 저장되도록 코드 수정.
                    # 그래서 2D image마다 contours를 찾아서 저장하고 있음, 이를 수정해야함.
                    rect =  cv2.boundingRect(contours[0])
                    dummy_var.append([j, rect])
                else:
                    dummy_rect = [0,0,0,0]
                    dummy_var.append([j, dummy_rect])

        all_masks.append(dummy_var)
    return all_masks


# mask_cn의 예상 입력은 file_paths이다. 
# sliced_masks_cn = compute_all_masks(var1, var2, mask_cn)
# sliced_masks_ad = compute_all_masks(var1, var2, mask_ad)


brats_sliced_masks = compute_all_masks(var1, var2, brats_mask_paths)

''' Select mask dimension and markers for Hippocampus region
'''
def markers(slices_mask):
    marks = []
    for i in range(len(slices_mask)):
        x_min = []
        y_min = []
        w_max = []
        h_max = []
        z_mn = 0
        indx_diff = []
        for j in range(len(slices_mask[i])):
            x_min.append(slices_mask[i][j][1][0])
            y_min.append(slices_mask[i][j][1][1])
            w_max.append(slices_mask[i][j][1][2])
            h_max.append(slices_mask[i][j][1][3])

            if slices_mask[i][j][1][0] != 0:
                indx_diff.append(slices_mask[i][j][0])
        x_min = np.array(x_min)
        y_min = np.array(y_min)
        x_mn = np.min(x_min[np.nonzero(x_min)])
        y_mn = np.min(y_min[np.nonzero(y_min)])
        w_mx = np.array(w_max).max()
        h_mx = np.array(h_max).max()
        dim = [x_mn, y_mn, w_mx, h_mx]
        # print(dim)
        d_mx = indx_diff[-1] - indx_diff[0]
        # print(diff)
        z_mn = indx_diff[0]
        marks.append([x_mn, y_mn, z_mn, w_mx, h_mx , d_mx])
    return marks

sliced_markers_cn = markers(sliced_masks_cn)
sliced_markers_ad = markers(sliced_masks_ad)

''' Concat labels 1 - AD,  0 -  CN
'''
labels_ad = np.ones((len(sliced_markers_ad),1))
labels_cn = np.zeros((len(sliced_markers_cn),1))

label_cn = np.hstack([sliced_markers_cn, labels_cn])
label_ad = np.hstack([sliced_markers_ad, labels_ad])


class data_cleaning_concat():
    def __init__(self, data1, data2, label1, label2):
        self.data1 = data1
        self.data2 = data2
        self.label1 = label1
        self.label2 = label2

    def _index_values(self, dataset):
        index = []
        for i in range(len(dataset)):
            if len(dataset[i]) < 145:
                index.append(i)
        return index

    def _pop_list(self, dataset, label, index):
        for i in range(len(index)):
            dataset.pop(index[i])
            label.pop(index[i])
        return dataset, label

    def files_concat(self):
        concat_data = []
        concat_label = []

        data1_ , label1_ = self._pop_list(self.data1, self.label1, self._index_values(self.data1))
        data2_ , label2_ = self._pop_list(self.data2, self.label2, self._index_values(self.data2))

        for i in range(len(data1_)):
            concat_data.append(data1_[i])
            concat_label.append(label1_[i])

        for j in range(len(data2_)):
            concat_data.append(data2_[j])
            concat_label.append(label2_[j])

        return shuffle(concat_data, concat_label)




data_clean = data_cleaning_concat(sliced_all_files_ad, sliced_all_files_cn,
                                                list(label_ad) , list(label_cn))
total_data , total_label = data_clean.files_concat()

def cross_valid_splits(data, label, fold):
    fold_train = []
    fold_test = []
    fold_ytrain = []
    fold_ytest = []

    indxs1 = int(1*len(data)/5)
    indxs2 = int(2*len(data)/5)
    indxs3 = int(3*len(data)/5)
    indxs4 = int(4*len(data)/5)
    indxs5 = int(5*len(data)/5)

    rn_trn = []
    rn_tst = []
    if fold == 1:
        for i in range(0, indxs4):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])

        for j in range(indxs4, indxs5):
            fold_test.append(data[j])
            fold_ytest.append(label[j])

    elif fold == 2:
        for i in range(0, indxs3):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])
        for i in range(indxs4,indxs5):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])


        for j in range(indxs3, indxs4):
            fold_test.append(data[j])
            fold_ytest.append(label[j])

    elif fold == 3:
        for i in range(0, indxs2):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])
        for i in range(indxs3,indxs5):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])


        for j in range(indxs2, indxs3):
            fold_test.append(data[j])
            fold_ytest.append(label[j])


    elif fold == 4:
        for i in range(0, indxs1):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])
        for i in range(indxs2,indxs5):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])


        for j in range(indxs1, indxs2):
            fold_test.append(data[j])
            fold_ytest.append(label[j])


    elif fold == 5:
        for i in range(indxs1, indxs5):
            fold_train.append(data[i])
            fold_ytrain.append(label[i])

        for j in range(0, indxs1):
            fold_test.append(data[j])
            fold_ytest.append(label[j])


    return fold_train, fold_test, np.array(fold_ytrain), np.array(fold_ytest)


f_train1, f_test1, f_ytrain1, f_ytest1 = cross_valid_splits(total_data, total_label,1)

f_train2, f_test2, f_ytrain2, f_ytest2 = cross_valid_splits(total_data, total_label,2)

f_train3, f_test3, f_ytrain3, f_ytest3 = cross_valid_splits(total_data, total_label,3)

f_train4, f_test4, f_ytrain4, f_ytest4 = cross_valid_splits(total_data, total_label,4)

f_train5, f_test5, f_ytrain5, f_ytest5 = cross_valid_splits(total_data, total_label,5)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

# Data_Gen

In [4]:
import glob
import numpy as np
import random
import fnmatch
import os
from PIL import Image
from matplotlib import pyplot as plt
import PIL
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

from bs4 import BeautifulSoup
from medpy.io import load
import cv2
from collections import namedtuple, deque
import math
from sklearn.utils import shuffle
import torch

f_train1 = "path to Training samples"
f_ytrain1 = "path to Training labels"
f_test1 = "path to Test samples"
f_ytest1 = "path to Test labels"


class data_set(Dataset):
  def __init__(self, file_path, label, transform_train):
    ### Variable definition to be used in Data loader
    self.file_path = file_path
    self.label = label
    self.transform_train = transform_train
    sample = nib.load('/Users/choihanjun/Library/Mobile Documents/com~apple~CloudDocs/Downloads/2024-1/[강화학습]/Project/archive_BraTS2022/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_233/BraTS20_Training_233_t1.nii').get_fdata()
    self.shape = sample.shape
    self.var1 = 0 # min slice
    self.var2 = 150 # max slice
    
    # TODO: 이 부분은 brats data를 위한 코드로 수정해야함.
  def _load_brats_data(path):
    print('load_brats_data...')
    files_path = []
    mask_paths = []
    unique_id = []
    for file_path in glob(f'{path}/*/*.nii'):
        # t1 파일만 처리
        if re.search(r"_t1(?=\.)", file_path):
            files_path.append(file_path)
        # seg 파일만 처리
        if "seg" in file_path:  # segmentation mask file
            mask_paths.append(file_path)
        unique_id.append(str(int(file_path.split('_')[-2])))
        
    return sorted(files_path), sorted(mask_paths), sorted(list(set(unique_id)), key=lambda x: int(x))

  def _compute_brats_masks(mask_paths, var1, var2):
      print('compute_brats_masks...')
      all_masks = []
      for mask_path in tqdm(mask_paths):
          mask = nib.load(mask_path).get_fdata()  # 마스크 데이터 로드
          dummy_var = []
          for j in range(min(var2, mask.shape[-1])):
              if j <= var2:  # 주어진 범위 내의 슬라이스만 처리
                  msk = mask[:, :, j] / mask.max() * 255  # 슬라이스 정규화 및 스케일링(0 ~ 1)
                  # msk = cv2.resize(msk, (256, 256))  # 이미지 크기 조정
                  _, bw_img = cv2.threshold(msk.astype(np.uint8), 10, 255, cv2.THRESH_OTSU)  # 이진화
                  contours, _ = cv2.findContours(bw_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)  # 컨투어 찾기
                  
                  # 가장 큰 컨투어 찾기
                  max_area = 0
                  max_rect = (0, 0, 0, 0)
                  for contour in contours:
                      rect = cv2.boundingRect(contour)
                      area = rect[2] * rect[3]  # rectangle의 넓이 (width * height)
                      if area > max_area:
                          max_area = area
                          max_rect = rect
                  
                  # 가장 큰 컨투어의 사각형 정보 저장
                  if max_area > 0:
                      dummy_var.append([j, max_rect])
                  else:
                      dummy_rect = [0, 0, 0, 0]  # 컨투어가 없을 경우
                      dummy_var.append([j, dummy_rect])
          all_masks.append(dummy_var)
      
      np_all_masks = np.array(all_masks, dtype=object)
      print(np_all_masks.shape)
      return all_masks


  def _brats_markers(slices_mask):
      print('brats_markers...')
      marks = []
      for i in tqdm(range(len(slices_mask))):
          x_min = []
          y_min = []
          w_max = []
          h_max = []
          indx_diff = []
          for j in range(len(slices_mask[i])):
              x_min.append(slices_mask[i][j][1][0])
              y_min.append(slices_mask[i][j][1][1])
              w_max.append(slices_mask[i][j][1][2])
              h_max.append(slices_mask[i][j][1][3])

              if slices_mask[i][j][1][0] != 0:
                  indx_diff.append(slices_mask[i][j][0])
          x_min = np.array(x_min)
          y_min = np.array(y_min)
          x_mn = np.min(x_min[np.nonzero(x_min)])
          y_mn = np.min(y_min[np.nonzero(y_min)])
          w_mx = np.array(w_max).max()
          h_mx = np.array(h_max).max()
          dim = [x_mn, y_mn, w_mx, h_mx]
          d_mx = indx_diff[-1] - indx_diff[0]
          z_mn = indx_diff[0]
          marks.append([x_mn, y_mn, z_mn, w_mx, h_mx, d_mx])
      return marks

  def __len__(self):
    ### size of file path
    return len(self.file_path)

  def __getitem__(self, indx):
    '''
      Read the image given indx from len function
    '''
    # image_input = np.ones((145,256,256))
    image_input = np.ones(self.shape) # (240, 240, 155 - 5)
    #for j in range(indx):
    for i in range(image_input[-1]):
        try:
            img, _ = load(self.file_path[indx][i])
            # img = cv2.resize(img.squeeze(), (256, 256))
            # TODO : Transformation을 추가해야함.
            # img = self.transform_train(img.squeeze().astype('float'))
            image_input[:, :, i] = img
        except:
            print("Check Index: ", indx)
    # box_gd = np.array([np.array(self.label)[indx][0], np.array(self.label)[indx][1], np.array(self.label)[indx][2],
    #           np.array(self.label)[indx][3], np.array(self.label)[indx][4], np.array(self.label)[indx][5]])
    box_gd = np.array(self.label)[indx]


    label_ = np.array(self.label)[indx][6]
    return [image_input, box_gd, label_]



train_data = data_set(file_path = f_train1, label = f_ytrain1, transform_train = transforms.Compose([
                                                                                   transforms.ToTensor(),
                                                                                   transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                                                                   ]))

unseen_data = data_set(file_path = f_test1, label = f_ytest1, transform_train = transforms.Compose([
                                                                                   transforms.ToTensor(),
                                                                                   transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                                                                   ]))

dataloader_train = torch.utils.data.DataLoader(train_data, batch_size = 64,
                        shuffle= True, num_workers= 0)
dataloader_unseen = torch.utils.data.DataLoader(unseen_data, batch_size = 1,
                        shuffle= True, num_workers= 0)

# Training.py

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d,MaxUnpool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from skimage import util
from torchvision import transforms, models
from torch import optim
import torchvision
import math
from tqdm import tqdm
import numpy as np
from utils import *

def training_phase(dqn_model, dataloader):
    training_acc = []
    training_loss = []
    for epoch in range(100):
        train_loss = 0.0
        train_acc = 0.0
        total = 0.0
        for data in dataloader:
            images, box_gds, labels = data
            batches, depth, height, width = images.shape
            dqn_mask = []
            loss_batch = torch.tensor(np.ones(batches))
            reward_batch = torch.tensor(np.ones(batches))
            dqn_mask = torch.tensor(np.ones((batches,1,50,height,width)))
            for batch in range(batches):
                # TODO: x, y, z가 80~120, 60~150, 50~95로 시작하는데, 이거는 따로 우리의 데이터에 맞게 수정해야함.
                x_start = np.random.randint(80,120)
                y_start = np.random.randint(60,200-50)
                z_start = np.random.randint(50,145-50)
                w, h, d = 50, 60, 50
                start_state = [x_start, y_start, z_start, w, h, d]
                game = "continue"
                env = images[batch]
                # 3D Labeling으로 x, y, z, w, h ,d가 필요함.
                x_gd, y_gd, z_gd = box_gds[batch][0], box_gds[batch][1], box_gds[batch][2]
                w_gd, h_gd, d_gd = box_gds[batch][3], box_gds[batch][4], box_gds[batch][5]
                state_gd = [x_gd, y_gd, z_gd, w_gd, h_gd, d_gd]
                start_eps = 1
                end_eps = 0.004
                start_step = 20
                end_step = 5
                prev_state = start_state.copy()
                loss = 0
                rewards = 0
                for episodes in range(100):
                    eps = max(0,(start_eps/int(1 + episodes)   - end_eps))
                    step = max(end_step, start_step/int(1 + episodes) )

                    # TODO: crop_reshape 함수가 하는 역할을 이해하고 우리의 데이터에 맞게 수정해야함.
                    # crop_reshpe는 3D로 바꿔주는 함수, crop_reshape(env, x, y, z, w, h, d) -> env[x:x+w, y:y+h, z:z+d]
                    # 즉, 원래 목적은 shape이 맞지 않은 3D image를 interpolation을 통해서 3D로 확장했음.
                    # 우리의 데이터는 그럴 필요가 없기 때문에, 바로 넘겨줘도 될 듯 함.
                    # 다만 prev_state가 정확히 어떤 의미인지 잘 모르겟음
                    # prev_state : [x, y, z, w, h, d]
                    prev_patch = crop_reshape(env, prev_state[0],prev_state[1],prev_state[2],prev_state[3],prev_state[4],prev_state[5])
                    action = dqn_model.select_action(prev_patch, game, eps)
                    if action != 6:
                        nxt_state = dqn_model.next_state(prev_state, action, step)
                        # print(nxt_state)
                        nxt_patch = crop_reshape(env, nxt_state[0],nxt_state[1],nxt_state[2],nxt_state[3],nxt_state[4],nxt_state[5])
                        game, reward = dqn_model.compute_reward(nxt_state, prev_state, state_gd, 10)
                    else:
                        game = "END"
                        nxt_state = prev_state.copy()
                        nxt_patch = crop_reshape(env, nxt_state[0],nxt_state[1],nxt_state[2],nxt_state[3],nxt_state[4],nxt_state[5])
                        _, reward = dqn_model.compute_reward(nxt_state, prev_state, state_gd, 10)
                    #print(np.asarray(action))
                    dqn_model.store_transition(prev_patch, np.asarray(int(action)), reward, nxt_patch)

                    if dqn_model.memory.memory_counter >= 1000:
                        loss = dqn_model.optimize(1500)


                    prev_state = nxt_state.copy()
                    prev_patch = crop_reshape(env, nxt_state[0],nxt_state[1],nxt_state[2],nxt_state[3],nxt_state[4],nxt_state[5])
                    rewards += reward
                    #print("Episodes : ", episodes)
                    if game == "END" or episodes == 2000:
                        dqn_mask[batch] = nxt_patch.view(1, nxt_patch.shape[-3], nxt_patch.shape[-2],nxt_patch.shape[-1])
                        loss_batch[batch] = (loss/int(1 + episodes))
                        reward_batch[batch] = (rewards/int(1 + episodes))
                        break
                print("State Pred {:} , ground truth {:}, episodes {:}, batch {:}".format(nxt_state , np.array(state_gd), episodes, batch))
            try:
                loss_per_batch = loss_batch.numpy().detach()
            except:
                loss_per_batch = loss_batch

            # print(loss_per_batch.shape)
            print("Loss_Rl {:.4f} \tReward_RL {:.4f}".format (loss_per_batch.mean(), reward_batch.mean()))
        train_loss = train_loss/len(dataloader)
        training_loss.append(train_loss)
        train_accuracy = train_acc/total #100 *
        training_acc.append(train_accuracy)
        print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.6f}'.format(epoch, train_loss, train_accuracy))
    return training_acc, training_loss

t_phase1, l_phase1 = training_phase(my_dqn,  dataloader_train)

# Testing

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d,MaxUnpool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from skimage import util
from torchvision import transforms, models
from torch import optim
from sklearn.metrics import confusion_matrix
import cv2
import torchvision
import math
from sklearn.utils import shuffle
from tqdm import tqdm

def testing_phase(dqn_model,classifier_model, dataloader, criterion):
    testing_acc = []
    testing_loss = []
    pred_test = []
    pred_true = []
    final_box = []
    test_loss = 0.0
    test_acc = 0.0
    total = 0.0
    for data in dataloader:
        images, box_gds, labels = data
        batches, depth, height, width = images.shape
        loss_batch = torch.tensor(np.ones(batches))
        reward_batch = torch.tensor(np.ones(batches))
        dqn_mask = torch.tensor(np.ones((batches,1,50,height,width)))
        for batch in range(batches):
            x_start = np.random.randint(80,120)
            y_start = np.random.randint(60,200-50)
            z_start = np.random.randint(50,145-50)
            w, h, d = 50, 60, 50
            start_state = [x_start, y_start, z_start, w, h, d]
            game = "continue"
            env = images[batch]
            x_gd, y_gd, z_gd = box_gds[batch][0], box_gds[batch][1], box_gds[batch][2]
            w_gd, h_gd, d_gd = box_gds[batch][3], box_gds[batch][4], box_gds[batch][5]
            state_gd = [x_gd, y_gd, z_gd, w_gd, h_gd, d_gd]
            start_eps = 1
            end_eps = 0.004
            start_step = 20
            end_step = 5
            prev_state = start_state.copy()
            loss = 0
            rewards = 0
            for episodes in range(500):
                eps = max(0,(start_eps/int(1 + episodes)   - end_eps))
                step = max(end_step, start_step/int(1 + episodes) )

                prev_patch = crop_reshape(env, prev_state[0],prev_state[1],prev_state[2],prev_state[3],prev_state[4],prev_state[5])
                action = dqn_model.select_action_test(prev_patch)
                if action != 6:
                    nxt_state = dqn_model.next_state(prev_state, action, step)
                    # print(nxt_state)
                    nxt_patch = crop_reshape(env, nxt_state[0],nxt_state[1],nxt_state[2],nxt_state[3],nxt_state[4],nxt_state[5])
                    game, reward = dqn_model.compute_reward(nxt_state, prev_state, state_gd, 10)
                else:
                    game = "END"
                    nxt_state = prev_state.copy()
                    nxt_patch = crop_reshape(env, nxt_state[0],nxt_state[1],nxt_state[2],nxt_state[3],nxt_state[4],nxt_state[5])
                    _, reward = dqn_model.compute_reward(nxt_state,prev_state, state_gd, 10)
                    #print(np.asarray(action))
                prev_state = nxt_state.copy()
                prev_patch = crop_reshape(env, nxt_state[0],nxt_state[1],nxt_state[2],nxt_state[3],nxt_state[4],nxt_state[5])
                rewards += reward
                #print("Episodes : ", episodes)
                if game == "END" or episodes == 500:
                    dqn_mask[batch] = nxt_patch.view(1, nxt_patch.shape[-3], nxt_patch.shape[-2],nxt_patch.shape[-1])
                    loss_batch[batch] = (loss/int(1 + episodes))
                    reward_batch[batch] = (rewards/int(1 + episodes))
                    break
            print("State Pred {:} , ground truth {:}, episodes {:}, batch {:}".format(nxt_state , np.array(state_gd), episodes, batch))
        try:
            loss_per_batch = loss_batch.numpy().detach()
        except:
            loss_per_batch = loss_batch

        # print(loss_per_batch.shape)
        print("Loss_Rl {:.4f} \tReward_RL {:.4f}".format (loss_per_batch.mean(), reward_batch.mean()))
        out_pred = classifier_model(dqn_mask.float(), images.view(batches, 1, depth, height, width).float())
        loss = criterion(out_pred, labels.type(torch.LongTensor))#.view(-1,1) float()
        test_loss += loss.item()
        total += labels.size(0)
        test_acc +=  binary_acc(out_pred, labels)
        pred_test.append(int(out_pred.detach()))
        pred_true.append(int(labels))
        final_box.append([nxt_state, np.array(state_gd)])
    test_loss = test_loss/len(dataloader)
    testing_loss.append(test_loss)
    test_acc = test_acc/total #100 *
    testing_acc.append(test_acc)
    print('Test Loss: {:.6f} \tTest Accuracy: {:.6f}'.format( test_loss, test_acc))
    return pred_test, final_box, test_loss, test_acc, pred_true


a,b,c,d,e = testing_phase(my_dqn, class_model,  dataloader_unseen, criterion )


def bar_plot(output_pred, output_true, unseen_loss, unseen_acc):
  plt.figure(2)
  cm = confusion_matrix(np.array(output_pred).squeeze(), np.array(output_true).squeeze())
  recall = 100*cm[0,0]/(cm[0,0] + cm[1,0])
  precision = 100*cm[0,0]/(cm[0,0] + cm[0,1])
  params = ['Loss', 'Accuracy', 'TN', 'FP', 'FN', 'TP', 'Precision', 'Recall']
  results = [unseen_loss, unseen_acc,cm[1,1],cm[0,1],cm[1,0],cm[0,0], precision, recall]
  plt.bar(params, results)
  for index,data in enumerate(results):
    plt.text(x = index -0.2 , y = data  , s = f"{int(data)}" , fontdict=dict(fontsize=10))
  plt.tight_layout()
  plt.show()

bar_plot(a,e,c,100*d)